**Fill in the gaps marked with "..." and answer all questions at the end of the notebook.**

# Exam question

Consider 10 cities which have the following annual demand for electrical energy:

city (i)|1|2|3|4|5|6|7|8|9|10
--------|-|-|-|-|-|-|-|-|-|--
demand $d_i$|30|18|21|10|8|23|11|32|14|9

There are 3 nearby energy production facilities which can serve any of the aforementioned cities. However, each city must be served by only one facility.

Activating a facility incurs the following fixed cost:

energy facility (j)|1|2|3
-------------------|-|-|-
activation cost $b_j$|80|100|110

When active, facilities can operate in one out of 4 possible production levels (annual capacities). For each production level, the capacity (in units of energy produced per year) and the respective total cost are summarized in the following table:

production level (k)|1|2|3|4
--------------------|-|-|-|-
capacity $l_k$|10|60|90|130
total cost $c_k$|100|480|630|780

The problem consists in assigning cities to facilities and selecting the production levels of the latter so that city demands are satisfied while minimizing the total facility costs.



## Solution

### Sets:
*   I is the set of cities.
*   J is the set of energy facilities.
*   K is the set of production levels.

### Parameters:
*   $d_i$ is the annual energy demand of city $i \in I$.
*   $b_j$ is the activation cost of facility $j \in J$.
*   $l_k$ is the amount of energy produced annually (capacity) at production level $k \in K$.
*   $c_k$ is the total cost of production level k.

### Variables:
*   $x_{ij} \in \{0, 1\}$ equals one if city i is assigned to facility j, 0 otherwise.
*   $y_j \in \{0, 1\}$ equals one if facility j is activated, 0 otherwise.
*   $z_{jk} \in \{0, 1\}$ equals one if facility j operates at production level k, 0 otherwise.

### Formulation:

$$
\begin{array}{lll}
\min & \sum_{j \in J} (b_j y_j + \sum_{k \in K}{c_k z_{jk}}  )\\
\textrm{s.t.} & \sum_{j \in J} x_{ij} = 1 & \forall i \in I \\
              & \sum_{i \in I} d_i x_{ij} \le \sum_{k \in K} l_k z_{jk} & \forall j \in J\\
              & y_j \ge \sum_{k \in K} z_{jk} & \forall j \in J\\
              & x_{ij}, y_j, z_{jk} \in \{0,1\}   & \forall i \in I, j \in J, k \in K
\end{array}
$$

In [ ]:
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

In [ ]:
import mip

n = 10 # number of cities
f =  3 # number of facilities
k =  4 # number of production levels

I = range(n)
J = range(f)
K = range(k)

d = [30, 18, 21, 10, 8, 23, 11, 32, 14, 9]
b = [80, 100, 110]
l = [10, 60, 90, 130]
c = [100, 480, 630, 780]

m = mip.Model()

x = [[m.add_var(var_type=mip.BINARY) for j in J] for i in I]
y = [m.add_var(var_type=mip.BINARY) for j in J]
z = [[m.add_var(var_type=mip.BINARY) for k in K] for j in J]

for i in I:
  m.add_constr(mip.xsum(x[i]) == 1)

for j in J:
  m.add_constr(mip.xsum(d[i] * x[i][j] for i in I) <= mip.xsum(l[k] * z[j][k] for k in K))

for j in J:
  m.add_constr(y[j] >= mip.xsum(z[j]))

m.objective = mip.minimize(
    mip.xsum(b[j] * y[j] for j in J) 
  + mip.xsum(c[k] * z[j][k] for k in K for j in J)
)

m.optimize()

print(m.objective_value)

for i in I:
  print([x[i][j].x for j in J])

Questions:

Suppose that we change the domain of $x_{ij}$ variables from
$$
x_{ij} \in \{0,1\} \qquad \forall i \in I, j \in J
$$
to
$$
x_{ij} \in [0,1] \subset \mathbb{R} \qquad \forall i \in I, j \in J
$$

1. Does the objective function value increases, decreases or remains the same? Why?

**Answer: it remains the same. Variables x do not appear in the objective function and a change in their domain does not affect the feasible region of the problem in such a way that the objective function value changes. This can be verified with code by changing the declaration of x variables (see code below).**

2. How does the assignment of cities to facilities changes? In particular, what happens to city $i=6$? 

**Answer: a city can now be assigned to multiple facilities. In particular, city 6 is served by facility 1 and 2.**

In [ ]:
x = [[m.add_var(lb=0, ub=1) for j in J] for i in I]